Mustafa Yetişir  

11/13/2022

In [7]:
from typing import List, Tuple, Any
import numpy as np
from PIL.Image import Image as ImageType
from PIL import Image

from utils import array_to_image, image_to_array
from renderer import noise_renderers

# Each pixel in the image must be given the change to be at the center of the filter

def apply_filter(image: ImageType, kernel: np.ndarray, padding: List[List[int]]) -> np.ndarray:
    """ 
    Apply a filter with the given kernel to the zero padded input image.
        **Note:** Kernels can be rectangular.
        **Note:** You can use ```np.meshgrid``` and indexing to avoid using loops (bonus +5) for convolving.
        **Do not** use ```np.convolve``` in this question.
        **Do not** use ```np.pad```. Use index assignment and slicing with numpy and do not loop
            over the pixels for padding.

    Args:
        image (ImageType): 2D Input image
        kernel np.ndarray: 2D kernel array of odd edge sizes
        padding: List[list[int]]: List of zero paddings. Example: [[3, 2], [1, 4]]. The first list
            [3, 2] determines the padding for the width of the image while [1, 4] determines the
            padding to apply to top and bottom of the image. The resulting image will have a shape
            of ((1 + H + 4), (3 + W + 2)).

    Raises:
        ValueError: If the length of kernel edges are not odd

    Returns:
        np.ndarray: Filtered array (May contain negative values)
    """

    img = image_to_array(image)

    def __check_kernel():
        res = np.asarray(kernel.shape) % 2 == 0
        if True in res:
            raise ValueError("Kernel edges are not odd.")

    def __pad_image():

        pad_top, pad_bottom = padding[1]
        pad_left, pad_right = padding[0]

        padded_img = np.zeros((img.shape[0]+pad_top+pad_bottom, img.shape[1]+pad_left+pad_right))
        padded_img[pad_top:pad_top+img.shape[0], pad_left:pad_left+img.shape[1]] = img

        return padded_img

    __check_kernel()
    __pad_image()

    step_horizontal = img.shape[0] - kernel.shape[0] + 1  #yatay
    step_vertical = img.shape[1] - kernel.shape[1] + 1    #dikey

    final_img = np.zeros((step_horizontal,step_vertical))

    for i in range(step_horizontal):
        for j in range(step_vertical):
            sub_img = img[i:i+kernel.shape[0], j:j+kernel.shape[1]]
            final_img[i][j] = round(np.sum(np.multiply(sub_img,kernel)))

    return final_img

def box_filter(image: ImageType, kernel_size: Tuple[int]) -> ImageType:
    """ Apply Box filter.

    Args:
        image (ImageType): 2D Input image of shape (H, W)
        kernel_size (Tuple[int]): 2D kernel size of kernel (height, width)

    Returns:
        ImageType: Filtered Image
    """

    box_kernel = np.ones(kernel_size) * 1/(kernel_size[0]*kernel_size[1])

    def __find_padding():
        pad_horizontal = kernel_size[1] // 2
        pad_vertical = kernel_size[0] // 2
        return [[pad_horizontal,pad_horizontal],[pad_vertical,pad_vertical]]

    padding = __find_padding()

    new_img = apply_filter(image, box_kernel, padding)
    return array_to_image(new_img)
    
def gaussian_filter(image: ImageType, kernel_size: Tuple[int], sigma: float) -> ImageType:
    """ Apply Gauss filter that is centered and has the shared standard deviation ```sigma```
    **Note:** Remember to normalize kernel before applying.
    **Note:** You can use ```np.meshgrid``` (once again) to generate Gaussian kernels

    Args:
        image (ImageType): 2D Input image of shape (H, W)
        kernel_size (Tuple[int]): 2D kernel size
        sigma (float): Standard deviation

    Returns:
        ImageType: Filtered Image
    """
    
    def __construct_kernel():
        kernel = np.zeros(kernel_size)
        for i in range(kernel_size[0]):
            for j in range(kernel_size[1]):
                kernel[i][j] = np.exp(-0.5*((i-kernel_size[0]//2)**2+(j-kernel_size[1]//2)**2)/(sigma**2))
        return kernel / np.sum(kernel)

    def __find_padding():
        pad_horizontal = kernel_size[1] // 2
        pad_vertical = kernel_size[0] // 2
        return [[pad_horizontal,pad_horizontal],[pad_vertical,pad_vertical]]

    padding = __find_padding()
    g_kernel = __construct_kernel()

    new_img = apply_filter(image, g_kernel, padding)
    return array_to_image(new_img)

In [8]:
# Test your above functions before running this cell
image = Image.open("noisy_image.png")
noise_renderers(image, gaussian_filter, box_filter)

Box filter is faster to calculate. Gaussian filter gives less importance to the further neighboring pixels. In gaussian filter sigma is more effective than kernel size for the blurring of the image. 

In [9]:
from renderer import edge_renderers

def horizontal_derivative(image: ImageType) -> ImageType:
    """ Return the horizontal derivative image with same padding.
    **Note**: Pad the input image so that the output image has the same size/shape.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        ImageType: Derivative image of shape (H, W).
    """
    img = image_to_array(image)

    padding = [[0,0],[0,0]]
    kernel = np.asarray([[-1],[0],[1]])
    

    new_img = apply_filter(image, kernel, padding)
    pad_x = (img.shape[0] - new_img.shape[0]) // 2
    pad_y = (img.shape[1] - new_img.shape[1]) // 2

    final_img = np.zeros(img.shape)

    final_img[pad_x:pad_x+new_img.shape[0], pad_y:pad_y+new_img.shape[1]] = new_img

    return array_to_image(final_img)


def vertical_derivative(image: ImageType) -> ImageType:
    """ Return the vertical derivative image with same padding.
    **Note**: Pad the input image so that the output image has the same size/shape.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        ImageType: Derivative image of shape (H, W).
    """
    img = image_to_array(image)

    padding = [[0,0],[0,0]]
    kernel = np.asarray([[-1,0,1]])
    

    new_img = apply_filter(image, kernel, padding)
    pad_x = (img.shape[0] - new_img.shape[0]) // 2
    pad_y = (img.shape[1] - new_img.shape[1]) // 2

    final_img = np.zeros(img.shape)

    final_img[pad_x:pad_x+new_img.shape[0], pad_y:pad_y+new_img.shape[1]] = new_img

    return array_to_image(final_img)


In [10]:
# Test your above functions before running this cell
image = Image.open("building.png")
edge_renderers(
    (image, "Original Image"),
    (vertical_derivative(image), "Vertical"),
    (horizontal_derivative(image), "Horizontal"),
)

I implemented Sobel filter for edge detection using 3x3 kernels.

Then, combined the output of the vertical and horizontal Sobel operators, namely $S_x$ and $S_y$, to obtain gradient image.
 

In [11]:
def sobel_vertical(image: ImageType) -> np.ndarray:
    """ Return the output of the vertical Sobel operator with same padding.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        np.ndarray: Derivative array of shape (H, W).
    """
    img = image_to_array(image)

    padding = [[0,0],[0,0]]
    kernel = np.asarray([[-1,0,1],[-2,0,2],[-1,0,1]])

    new_img = apply_filter(image, kernel, padding)
    pad_x = (img.shape[0] - new_img.shape[0]) // 2
    pad_y = (img.shape[1] - new_img.shape[1]) // 2

    final_img = np.zeros(img.shape)

    final_img[pad_x:pad_x+new_img.shape[0], pad_y:pad_y+new_img.shape[1]] = new_img

    return array_to_image(final_img)


def sobel_horizontal(image: ImageType) -> np.ndarray:
    """ Return the output of the horizontal Sobel operator with same padding.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        np.ndarray: Derivative array of shape (H, W).
    """
    img = image_to_array(image)

    padding = [[0,0],[0,0]]
    kernel = np.asarray([[-1,-2,-1],[0,0,0],[1,2,1]])

    new_img = apply_filter(image, kernel, padding)
    pad_x = (img.shape[0] - new_img.shape[0]) // 2
    pad_y = (img.shape[1] - new_img.shape[1]) // 2

    final_img = np.zeros(img.shape)

    final_img[pad_x:pad_x+new_img.shape[0], pad_y:pad_y+new_img.shape[1]] = new_img

    return array_to_image(final_img)


def gradient_image(image: ImageType) -> ImageType:
    """ Return the gradient image calculated by combining the output of Sobel filters.

    Args:
        image (ImageType): 2D Input Image of shape (H, W)

    Returns:
        ImageType: Derivative image of shape (H, W).
    """
    sobel_v = sobel_vertical(image)
    sobel_h = sobel_horizontal(image)

    sobel_v = image_to_array(sobel_v)
    sobel_h = image_to_array(sobel_h)

    return array_to_image(sobel_v + sobel_h)

image = Image.open("building.png")
edge_renderers(
    (image, "Original Image"),
    (gradient_image(image), "Edge Image"),
)

In [12]:
# Test your above functions before running this cell
image = Image.open("building.png")
edge_renderers(
    (image, "Original Image"),
    (gradient_image(image), "Edge Image"),
)